# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
# Snir Taub - 207332107
# Details Student 2:


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [ ]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [ ]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [ ]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
def tokenizer(hebrew_string):
    tokens_with_labels = ht.tokenize(hebrew_string)
    hebrew_tokens = [token_info[1] for token_info in tokens_with_labels if token_info[0] == 'HEBREW']
    return hebrew_tokens


In [12]:
# Mapping gender values to binary values
gender_mapping = {'m': 1, 'f': 0}

# Applying the mapping to the 'gender' column
df_train['gender'] = df_train['gender'].map(gender_mapping)

In [13]:
# Splitting the training dataset into feature vectors and target labels
X_train = df_train['story']   # Extracting the 'story' column as feature vectors
y_train = df_train['gender']  # Extracting the 'gender' column as target labels

In [14]:
X_train.head()
y_train.head()

0    כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...
1    לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...
2    מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...
3    כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...
4    ‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...
Name: story, dtype: object

0    1
1    1
2    0
3    1
4    0
Name: gender, dtype: int64

In [16]:
# Define cross-validation parameters
cv = 10

# Define shared classifier and hyperparameters
shared_classifier = SGDClassifier(max_iter=30, random_state=42, alpha=0.0001, penalty='l1', loss='hinge',
                                  learning_rate='optimal', eta0=1, validation_fraction=0.1, n_iter_no_change=5)

# Define shared preprocessing steps
preprocessor = preprocessing.Normalizer(norm='l2')

# Define vectorizers and their parameter grids
vectorizers = [
    ("CountVectorizer", CountVectorizer(tokenizer=tokenizer, max_df=0.75, ngram_range=(1, 3))),
    ("TfidfVectorizer", TfidfVectorizer(tokenizer=tokenizer, max_df=0.75, ngram_range=(1, 3)))
]

for vectorizer_name, vectorizer in vectorizers:
    print(f"Evaluating {vectorizer_name}...")
    
    # Create pipeline for the current vectorizer
    pipeline = Pipeline([
        (vectorizer_name.lower(), vectorizer),
        ('normalize', preprocessor),
        ('clf', shared_classifier),
    ])
    
    # Fit the pipeline on the training data
    pipeline.fit(X_train, y_train)
    
    # Evaluate using cross-validation
    f1_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='f1_macro')
    avg_f1_score = f1_scores.mean()
    
    print(f"{vectorizer_name} - Average F1 score: {avg_f1_score}")

Evaluating CountVectorizer...


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_df=0.75, ngram_range=(1, 3),
                                 tokenizer=<function tokenizer at 0x00000216C2367160>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=1, max_iter=30, penalty='l1',
                               random_state=42))])

CountVectorizer - Average F1 score: 0.6711131103633516
Evaluating TfidfVectorizer...


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.75, ngram_range=(1, 3),
                                 tokenizer=<function tokenizer at 0x00000216C2367160>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=1, max_iter=30, penalty='l1',
                               random_state=42))])

TfidfVectorizer - Average F1 score: 0.7281497183495187


In [17]:
# Create pipelines for different models
pipelines = [
    
    ('SGDClassifier', Pipeline([
        ('tfidf', TfidfVectorizer(analyzer=tokenizer)),
        ('normalize', preprocessing.Normalizer(norm='l2')),
        ('clf', SGDClassifier(max_iter=1000, random_state=42))
    ]), {
        'tfidf__max_df': [0.5, 0.75, 1.0],
        'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
        'clf__penalty': ['l1', 'l2', 'elasticnet'],
        'clf__max_iter': [1000, 2000, 3000]
    }),

    ('Naive Bayes', Pipeline([
        ('tfidf', TfidfVectorizer(analyzer=tokenizer)),
        ('normalize', preprocessing.Normalizer(norm='l2')),
        ('clf', MultinomialNB())
    ]), {
        'tfidf__max_df': [0.5, 0.75, 1.0],
        'clf__alpha': [0.0001, 0.001, 0.01, 0.1]
    }),

    ('Linear SVC', Pipeline([
        ('tfidf', TfidfVectorizer(analyzer=tokenizer)),
        ('normalize', preprocessing.Normalizer(norm='l2')),
        ('classifier', LinearSVC())
    ]), {
        'tfidf__max_df': [0.5, 0.75, 1.0],
        'classifier__C': [0.1, 1, 10],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__max_iter': [1000, 2000, 3000]
    })

]

best_model = None
best_f1_score = 0


In [18]:
# Loop over the pipelines and perform grid search with cross-validation
for model_name, pipeline, param_grid in pipelines:
    print(f"Evaluating {model_name}")
    grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='f1_macro')
    grid_search.fit(X_train, y_train)
    
    # Print best hyperparameters and CV score
    print(f"Best hyperparameters for {model_name}: {grid_search.best_params_}")
    print(f"Cross-validation F1 score for {model_name}: {grid_search.best_score_}")
    print("------------------------")
    # Update best model if necessary
    if grid_search.best_score_ > best_f1_score:
        best_f1_score = grid_search.best_score_
        best_model = grid_search.best_estimator_

# Print the best model and its F1 score
print(f"Best model: {best_model.named_steps['clf']}")
print(f"Best F1 score: {best_f1_score}")

Evaluating SGDClassifier


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer=<function tokenizer at 0x00000216C2367160>)),
                                       ('normalize', Normalizer()),
                                       ('clf',
                                        SGDClassifier(random_state=42))]),
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'clf__max_iter': [1000, 2000, 3000],
                         'clf__penalty': ['l1', 'l2', 'elasticnet'],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for SGDClassifier: {'clf__alpha': 0.0001, 'clf__max_iter': 1000, 'clf__penalty': 'l1', 'tfidf__max_df': 0.75}
Cross-validation F1 score for SGDClassifier: 0.710115679911672
------------------------
Evaluating Naive Bayes


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer=<function tokenizer at 0x00000216C2367160>)),
                                       ('normalize', Normalizer()),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for Naive Bayes: {'clf__alpha': 0.01, 'tfidf__max_df': 0.5}
Cross-validation F1 score for Naive Bayes: 0.5075329955950617
------------------------
Evaluating Linear SVC


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer=<function tokenizer at 0x00000216C2367160>)),
                                       ('normalize', Normalizer()),
                                       ('classifier', LinearSVC())]),
             param_grid={'classifier__C': [0.1, 1, 10],
                         'classifier__max_iter': [1000, 2000, 3000],
                         'classifier__penalty': ['l1', 'l2'],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for Linear SVC: {'classifier__C': 10, 'classifier__max_iter': 1000, 'classifier__penalty': 'l2', 'tfidf__max_df': 0.75}
Cross-validation F1 score for Linear SVC: 0.6416167516947804
------------------------
Best model: SGDClassifier(penalty='l1', random_state=42)
Best F1 score: 0.710115679911672


In [19]:
# Fit the best model on the training data
best_model.fit(X_train, y_train)

# Perform cross-validation to evaluate model performance
# using F1 score with macro averaging on 10 folds
f1_scores = cross_val_score(best_model, X_train, y_train, cv=10, scoring='f1_macro')

# Calculate the average F1 score across all folds
average_f1_score = np.mean(f1_scores)

# Print the average F1 score
print("Average F1 score:", average_f1_score)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer=<function tokenizer at 0x00000216C2367160>,
                                 max_df=0.75)),
                ('normalize', Normalizer()),
                ('clf', SGDClassifier(penalty='l1', random_state=42))])

Average F1 score: 0.710115679911672


In [20]:
# Predict gender labels using the best model on the test data
predicted_labels = best_model.predict(df_test["story"])

# Map numerical labels (1 for male, 0 for female) to textual labels ('m' for male, 'f' for female)
predicted_gender = np.where(predicted_labels == 1, 'm', 'f')

# Create a DataFrame to store the results, including test example IDs and predicted gender labels
results_df = pd.DataFrame({'id_test': df_test["test_example_id"], 'y_pred': predicted_gender})

# Display the first 5 rows of the results
print("First 5 predictions:")
print(results_df.head())

# Display the last 5 rows of the results
print("Last 5 predictions:")
print(results_df.tail())

First 5 predictions:
   id_test y_pred
0        0      m
1        1      m
2        2      m
3        3      m
4        4      f
Last 5 predictions:
     id_test y_pred
318      318      m
319      319      m
320      320      m
321      321      m
322      322      m


In [21]:
# Create a DataFrame to store the predicted results,
# including test example IDs and predicted gender labels
df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_gender': predicted_gender})

# Display the DataFrame containing the predicted results
print(df_predicted)

     test_example_id predicted_gender
0                  0                m
1                  1                m
2                  2                m
3                  3                m
4                  4                f
..               ...              ...
318              318                m
319              319                m
320              320                m
321              321                m
322              322                m

[323 rows x 2 columns]


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [22]:
df_predicted.to_csv('classification_results.csv',index=False)